In [1]:
import pandas as pd
import pyarrow.parquet as pq


In [2]:
import pandas as pd
import os

# Especifica la ruta de la carpeta que contiene los archivos .parquet particionados
ruta_carpeta_parquet = '../datasets/parquets/metadatos Google'

# Lista para almacenar DataFrames de cada partición
dataframes = []

# Itera sobre los archivos .parquet en la carpeta
for root, dirs, files in os.walk(ruta_carpeta_parquet):
    for file in files:
        if file.endswith('.parquet'):
            # Construye la ruta completa del archivo
            ruta_completa = os.path.join(root, file)
            
            # Lee el archivo .parquet y agrega el DataFrame a la lista
            df_particion = pd.read_parquet(ruta_completa)
            dataframes.append(df_particion)

# Concatena todos los DataFrames en uno solo
df_google = pd.concat(dataframes, ignore_index=True)

In [3]:
# Explode the "categories" column
df_explode = df_google.explode('category')

# Reset the index if needed
df_explode.reset_index(drop=True, inplace=True)

In [4]:
categorias = pd.read_csv("..\datasets\parquets\Categorias-Google.csv")

In [6]:
# Extract unique values from the "category" column in categorias DataFrame
valid_categories = categorias['category']

# Filter df_google based on valid categories
#df_google = df_google[df_google['category'].isin(valid_categories)]
df_explode = df_explode[df_explode['category'].isin(valid_categories)] 
df_google=  df_google[df_google['gmap_id'].isin(df_explode['gmap_id'].unique())]  
# Reset the index if needed
df_google.reset_index(drop=True, inplace=True)



In [7]:
df_google.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,State
0,Polished Nail Salon & Spa,"Polished Nail Salon & Spa, 1330 Hickman Rd, Wa...",0x87ec238afdd8837b:0x2f43f2b12142bff5,None,41.614198,-93.847269,"[Nail salon, Waxing hair removal service]",4.0,86,None,"[[Thursday, 9AM–8PM], [Friday, 9AM–8PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 8PM,"[0x87ec2481df68e107:0x2978d64e14ca8313, 0x87ec...",https://www.google.com/maps/place//data=!4m2!3...,IA
1,Lena’s Beauty Nails Lashes Brows,"Lena’s Beauty Nails Lashes Brows, 657 NE Ventu...",0x87ec2481df6626b5:0xff5627745308ec2b,None,41.616310,-93.858345,[Beauty salon],4.2,128,None,"[[Thursday, 9AM–7PM], [Friday, 9AM–7PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 7PM,"[0x87ec20ce836cac57:0x83bd66577e9a7a32, 0x87ee...",https://www.google.com/maps/place//data=!4m2!3...,IA
2,Angel-A Massage,"Angel-A Massage, 5329 Dewey Dr, Fair Oaks, CA ...",0x809adef58ae560bd:0x83412889e022ff70,None,38.662630,-121.308639,"[Massage spa, Chiropractor, Massage, Massage t...",4.3,6,None,"[[Thursday, 9:30AM–9:30PM], [Friday, 9:30AM–9:...",None,Open ⋅ Closes 9:30PM,"[0x809adee9ac6f1bd7:0xe2e56d16919b9b8d, 0x809a...",https://www.google.com/maps/place//data=!4m2!3...,CA
3,Elite Massage,"Elite Massage, 8121 Madison Ave, Fair Oaks, CA...",0x809ae0a7ce91433b:0xe4b44fbb8c35ba57,None,38.664915,-121.260192,[Massage therapist],5.0,1,None,None,None,None,"[0x809ade27605defc1:0xdedc5626a281fd93, 0x809a...",https://www.google.com/maps/place//data=!4m2!3...,CA
4,Happy Spa & Massage (Formerly Hawaii Massage &...,Happy Spa & Massage (Formerly Hawaii Massage &...,0x809b217a6cbe4fa9:0x9fd06999c64c1815,None,38.785548,-121.286886,"[Massage therapist, Massage spa, Spa]",2.7,8,None,"[[Thursday, 9:30AM–10PM], [Friday, 9:30AM–10PM...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 10PM,"[0x809b20410d8df0fb:0x60610399a7ec1ba6, 0x809b...",https://www.google.com/maps/place//data=!4m2!3...,CA


In [8]:
df_google.shape

(210781, 16)

In [16]:
df_google.isnull().sum()/df_google.shape[0]*100

name                 0.000000
address              0.823129
gmap_id              0.000000
description         92.312400
latitude             0.000000
longitude            0.000000
category             0.000000
avg_rating           0.000000
num_of_reviews       0.000000
price               95.368178
hours               13.448556
MISC                14.543057
state               12.804759
relative_results     6.071230
url                  0.000000
State                0.307903
dtype: float64

Debido a la cantidad de nulos las columnas description y price serán eliminadas

In [23]:
# Iterar sobre las columnas y verificar si son hashables
non_hashable_columns = []

for col in df_google.columns:
    try:
        hash(df_google[col])
    except TypeError:
        non_hashable_columns.append(col)

# Imprimir las columnas no hashables
print("Columnas no hashables:", non_hashable_columns)

Columnas no hashables: ['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC', 'state', 'relative_results', 'url', 'State']


In [56]:
df_simplificado=df_google[['name','gmap_id','State',"num_of_reviews"]]

In [57]:
df_simplificado.drop_duplicates(inplace=True)

C:\Users\Grethel\AppData\Local\Temp\ipykernel_19392\3924905242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simplificado.drop_duplicates(inplace=True)


In [60]:
# Mostrar el DataFrame resultante
df_simplificado


,name,gmap_id,State,num_of_reviews
0,Polished Nail Salon & Spa,0x87ec238afdd8837b:0x2f43f2b12142bff5,IA,86
1,Lena’s Beauty Nails Lashes Brows,0x87ec2481df6626b5:0xff5627745308ec2b,IA,128
2,Angel-A Massage,0x809adef58ae560bd:0x83412889e022ff70,CA,6
3,Elite Massage,0x809ae0a7ce91433b:0xe4b44fbb8c35ba57,CA,1
4,Happy Spa & Massage (Formerly Hawaii Massage &...,0x809b217a6cbe4fa9:0x9fd06999c64c1815,CA,8
...,...,...,...,...
210776,Nails 7,0x8649c77ea5deee03:0x5bd1ca06d2f0c1ac,TX,108
210777,Scarlett O Haira's,0x88d929ef98b5ff25:0xa550550941e533f4,FL,18
210778,Townzen Barber Shop,0x87c91705fa79bec5:0x7a5d0c631376c543,AR,28
210779,"K. Renee Salon, Spas and Studios",0x8640c11d40bcda95:0x4771f905231fae90,TX,18


In [62]:
df_simplificado = df_simplificado.groupby(['name', 'gmap_id'])['num_of_reviews'].sum().reset_index()

In [49]:
df_simplificado.sort_values(by='num_of_reviews',ascending=False).head(100).to_csv(r'..\datasets\csv\top_100_reviews_google.csv',index=False)